# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [334]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [335]:
#Import file to pandas DF
csv_file = os.path.join('output_data', 'cities.csv')
city_weather = pd.read_csv(csv_file)
city_weather

,City,Latitude,Longitude,Max Temperature (degF),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date/Time (UTC)
0,Busselton,-33.65,115.33,57.99,56,32,20.33,AU,2020-06-18 03:19:28
1,Hobart,-42.88,147.33,46.99,76,42,1.99,AU,2020-06-18 03:23:15
2,Makakilo City,21.35,-158.09,84.20,51,75,6.93,US,2020-06-18 03:24:24
3,Tekeli,44.83,78.82,59.85,46,58,7.56,KZ,2020-06-18 03:24:24
4,Hithadhoo,-0.60,73.08,84.00,75,9,7.25,MV,2020-06-18 03:24:24
...,...,...,...,...,...,...,...,...,...
542,Alofi,-19.06,-169.92,77.00,57,40,16.11,NU,2020-06-18 03:25:32
543,Cururupu,-1.83,-44.87,73.63,95,18,1.63,BR,2020-06-18 03:26:44
544,Ryōtsu-minato,38.08,138.43,71.60,78,100,8.05,JP,2020-06-18 03:26:44
545,Christchurch,-43.53,172.63,46.99,100,100,9.17,NZ,2020-06-18 03:23:02


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [336]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [337]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather[['Latitude', 'Longitude']].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values

humidity = city_weather["Humidity (%)"].astype(float)

In [338]:
#Format figure
figure_layout = {
    'width': '1000px',
    'height': '550px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Generate Google Maps Figure
fig = gmaps.figure(layout=figure_layout,center=(50, -15), zoom_level=1.5)

#Create heat layer based upon the humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
#Add the newly created layer to the figure
fig.add_layer(heat_layer)

#Display the Figure
fig

Figure(layout=FigureLayout(height='550px', margin='0 auto 0 auto', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [340]:
#Create new DF based upon the city weather but drop all rows that dont have all three properties required for analysis

city_weather_clean = city_weather.dropna(subset=['Max Temperature (degF)', 'Wind Speed (mph)', 'Cloudiness (%)'])

In [354]:
#Specify Necessary conditions
city_weather_clean = city_weather_clean.loc[(city_weather_clean['Max Temperature (degF)'] < 80) &
                                            (city_weather_clean['Max Temperature (degF)'] > 70) &
                                            (city_weather_clean['Wind Speed (mph)'] < 10) &
                                            (city_weather_clean['Cloudiness (%)'] == 0)]

#Display newly created dataframe with maximum of 10 rows
city_weather_clean = city_weather_clean.iloc[:10]
city_weather_clean

,City,Latitude,Longitude,Max Temperature (degF),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date/Time (UTC)
35,Makat,47.65,53.35,79.29,36,0,8.30,KZ,2020-06-18 03:21:12
63,San Patricio,19.22,-104.70,79.32,66,0,4.07,MX,2020-06-18 03:24:36
78,São Filipe,14.90,-24.50,77.02,76,0,7.61,CV,2020-06-18 03:24:39
105,Gaza,31.50,34.47,72.00,59,0,1.99,PS,2020-06-18 03:24:45
162,Mossamedes,-15.20,12.15,71.51,52,0,5.82,AO,2020-06-18 03:24:58
353,Emba,48.83,58.14,72.39,48,0,9.80,KZ,2020-06-18 03:25:40
359,Sakakah,29.97,40.21,75.20,27,0,9.17,SA,2020-06-18 03:25:42
397,Archidona,37.10,-4.39,73.99,43,0,2.28,ES,2020-06-18 03:25:54
399,Adamantas,36.73,24.43,71.60,73,0,5.82,GR,2020-06-18 03:25:54
433,Tomatlán,19.93,-105.25,77.50,65,0,2.89,MX,2020-06-18 03:26:07


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [355]:
#Copy clean data to newly named DF
hotel_df = city_weather_clean
hotel_df

,City,Latitude,Longitude,Max Temperature (degF),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date/Time (UTC)
35,Makat,47.65,53.35,79.29,36,0,8.30,KZ,2020-06-18 03:21:12
63,San Patricio,19.22,-104.70,79.32,66,0,4.07,MX,2020-06-18 03:24:36
78,São Filipe,14.90,-24.50,77.02,76,0,7.61,CV,2020-06-18 03:24:39
105,Gaza,31.50,34.47,72.00,59,0,1.99,PS,2020-06-18 03:24:45
162,Mossamedes,-15.20,12.15,71.51,52,0,5.82,AO,2020-06-18 03:24:58
353,Emba,48.83,58.14,72.39,48,0,9.80,KZ,2020-06-18 03:25:40
359,Sakakah,29.97,40.21,75.20,27,0,9.17,SA,2020-06-18 03:25:42
397,Archidona,37.10,-4.39,73.99,43,0,2.28,ES,2020-06-18 03:25:54
399,Adamantas,36.73,24.43,71.60,73,0,5.82,GR,2020-06-18 03:25:54
433,Tomatlán,19.93,-105.25,77.50,65,0,2.89,MX,2020-06-18 03:26:07


In [356]:
#Create a list of zipped latitudes and longitudes for use in coordinate systems
lat_longs = list(zip(hotel_df.Latitude, hotel_df.Longitude))

In [357]:
#Initialize 4 new lists for future use
hotel_name = []
hotel_lat = []
hotel_lon = []
no_hotels_lat = []

#Specify parameters
for coord in lat_longs:
    # geocoordinates
    list(coord)
    target_radius = 5000
    keyword = "hotel"

# URL
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={coord[0]},\
          {coord[1]}&radius={target_radius}&keyword={keyword}&key={g_key}"

#Create JSON response object
    response = requests.get(url)
    response_json = response.json()
    
# Try and Except for hotel names and locations. In some cases no hotels can be found and that is taken into account.   
    try:
        hotel_name.append(response_json['results'][0]['name'])
        hotel_lat.append(round((response_json['results'][0]['geometry']['location']['lat']),4))
        hotel_lon.append(round((response_json['results'][0]['geometry']['location']['lng']),4))
        print(f'Found {(citipy.nearest_city(coord[0], coord[1]).city_name).title()}! Appending to list.')
    except:
        no_hotels_lat.append(coord[0])
        print(f"No hotel found for {(hotel_df.loc[hotel_df['Latitude'] == coord[0]]['City']).values}. Proceeding to next record.")  

Found Makat! Appending to list.
Found San Patricio! Appending to list.
Found Sao Filipe! Appending to list.
Found Sederot! Appending to list.
Found Namibe! Appending to list.
No hotel found for ['Emba']. Proceeding to next record.
Found Sakakah! Appending to list.
Found Archidona! Appending to list.
Found Adamas! Appending to list.
Found Tomatlan! Appending to list.


In [358]:
#For all cities where a hotel cannot be found, they need to be removed from the list of possible desinations.
# That can be done by finding the index of those rows
no_hotels_index = []
for latitude in no_hotels_lat:
    no_hotels_index.append(hotel_df.loc[hotel_df['Latitude'] == latitude].index.item())

In [359]:
#If no hotel found, remove from dataframe by index. Limit to 10 rows in case additional results are found.

clean_hotel_df = hotel_df.drop(no_hotels_index, axis=0)
clean_hotel_df

,City,Latitude,Longitude,Max Temperature (degF),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date/Time (UTC)
35,Makat,47.65,53.35,79.29,36,0,8.30,KZ,2020-06-18 03:21:12
63,San Patricio,19.22,-104.70,79.32,66,0,4.07,MX,2020-06-18 03:24:36
78,São Filipe,14.90,-24.50,77.02,76,0,7.61,CV,2020-06-18 03:24:39
105,Gaza,31.50,34.47,72.00,59,0,1.99,PS,2020-06-18 03:24:45
162,Mossamedes,-15.20,12.15,71.51,52,0,5.82,AO,2020-06-18 03:24:58
359,Sakakah,29.97,40.21,75.20,27,0,9.17,SA,2020-06-18 03:25:42
397,Archidona,37.10,-4.39,73.99,43,0,2.28,ES,2020-06-18 03:25:54
399,Adamantas,36.73,24.43,71.60,73,0,5.82,GR,2020-06-18 03:25:54
433,Tomatlán,19.93,-105.25,77.50,65,0,2.89,MX,2020-06-18 03:26:07


In [360]:
#Add new columns with hotel information and display the DataFrame
clean_hotel_df['Hotel Name'] = hotel_name
clean_hotel_df['Hotel Latitude'] = hotel_lat
clean_hotel_df['Hotel Longitude'] = hotel_lon
clean_hotel_df

,City,Latitude,Longitude,Max Temperature (degF),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date/Time (UTC),Hotel Name,Hotel Latitude,Hotel Longitude
35,Makat,47.65,53.35,79.29,36,0,8.30,KZ,2020-06-18 03:21:12,Gostinitsa Daulet,47.6509,53.3388
63,San Patricio,19.22,-104.70,79.32,66,0,4.07,MX,2020-06-18 03:24:36,La Paloma Oceanfront Retreat,19.2225,-104.7024
78,São Filipe,14.90,-24.50,77.02,76,0,7.61,CV,2020-06-18 03:24:39,Hotel Santos Pina,14.8927,-24.4981
105,Gaza,31.50,34.47,72.00,59,0,1.99,PS,2020-06-18 03:24:45,فندق جراند بالاس Grand Palace Hotel,31.5269,34.4375
162,Mossamedes,-15.20,12.15,71.51,52,0,5.82,AO,2020-06-18 03:24:58,Hotel ibis Styles Iu Namibe,-15.1909,12.1512
359,Sakakah,29.97,40.21,75.20,27,0,9.17,SA,2020-06-18 03:25:42,Raoum Inn Hotel,29.9453,40.1812
397,Archidona,37.10,-4.39,73.99,43,0,2.28,ES,2020-06-18 03:25:54,Hotel Escua,37.0927,-4.3962
399,Adamantas,36.73,24.43,71.60,73,0,5.82,GR,2020-06-18 03:25:54,Santa Maria Village Resort & Spa,36.7280,24.4555
433,Tomatlán,19.93,-105.25,77.50,65,0,2.89,MX,2020-06-18 03:26:07,Hotel Misión San Miguel,19.9419,-105.2483


In [361]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Hotel Latitude", "Hotel Longitude"]]

In [362]:
# Customize the size of the figure

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout,center=(50.0, +15), zoom_level=1.9)

# Add the layer to the map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(layout=figure_layout,center=(40.0, -5.0), zoom_level=1.9)
fig.add_layer(marker_layer)
#Base hotel_info map
fig

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', padding='1px', width='1000px'))

In [363]:
#Print combined map of hotel information and heat map
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout,center=(50.0, +15), zoom_level=1.9)

# Create Hotel info layer
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Specify Figure parameters
fig = gmaps.figure(layout=figure_layout,center=(40.0, -5.0), zoom_level=1.9)

#Add heat and marker layer to the figure
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

#Display the figure
fig

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', padding='1px', width='1000px'))